## Pre-collect all the data in this cell

In [82]:
import os
from typing import Optional

import requests
from dotenv import load_dotenv
from gql import Client
from gql.transport.requests import RequestsHTTPTransport
from web3 import Web3

load_dotenv()

ARB_CHAIN_ID = 42161
BALANCER_GAUGE_URL = "https://api.thegraph.com/subgraphs/name/balancer-labs/balancer-gauges"
BALANCER_GAUGE_CONTROLLER_ADDR = Web3.toChecksumAddress("0xC128468b7Ce63eA702C1f104D55A2566b13D3ABD")
BALANCER_GAUGE_CONTROLLER_ABI = [{"stateMutability": "view", "type": "function", "name": "gauge_relative_weight",
                                  "inputs": [{"name": "addr", "type": "address"}],
                                  "outputs": [{"name": "", "type": "uint256"}]},
                                 {"stateMutability": "view", "type": "function", "name": "gauge_relative_weight",
                                  "inputs": [{"name": "addr", "type": "address"}, {"name": "time", "type": "uint256"}],
                                  "outputs": [{"name": "", "type": "uint256"}]}]

web3 = Web3(Web3.HTTPProvider(os.environ["ETHNODEURL"]))


def make_gql_client(url: str) -> Optional[Client]:
    transport = RequestsHTTPTransport(url=url, retries=3)
    return Client(
        transport=transport, fetch_schema_from_transport=True, execute_timeout=60
    )


client = make_gql_client(BALANCER_GAUGE_URL)

# Fetch all voting gauges from github json
voting_gauges_req = requests.get(
    "https://raw.githubusercontent.com/balancer/frontend-v2/master/src/data/voting-gauges.json")
if not voting_gauges_req.ok:
    raise ValueError("Failed to fetch voting gauges")
voting_gauges = voting_gauges_req.json()

# Collect arb gauges
arb_gauges = {}
for gauge in voting_gauges:
    # Only collect gauges for the arb chain and that are not killed
    if int(gauge['network']) == ARB_CHAIN_ID and gauge['isKilled'] is False:
        arb_gauges[gauge['address']] = {
            'gaugeAddress': gauge['address'],
            'pool': gauge['pool']['address'],
            'symbol': gauge['pool']['symbol']
        }

gauge_c_contract = web3.eth.contract(address=BALANCER_GAUGE_CONTROLLER_ADDR, abi=BALANCER_GAUGE_CONTROLLER_ABI)

vote_weights = {}
# Collect gauge voting weights from the gauge controller on chain
for gauge_addr, gauge_data in arb_gauges.items():
    weight = gauge_c_contract.functions.gauge_relative_weight(gauge_addr).call() / 1e18 * 100
    vote_weights[gauge_addr] = weight
    arb_gauges[gauge_addr]['voteWeight'] = weight

## Calculate arbitrum distribution across gauges

In [83]:
from copy import deepcopy
from IPython.core.display import HTML
import pandas as pd
ARBITRUM_TO_DISTRIBUTE = 97404
VOTE_CAP_IN_PERCENT = 10  # 10% cap on any single gauge
VOTE_CAP = VOTE_CAP_IN_PERCENT / 100 * ARBITRUM_TO_DISTRIBUTE
# Calculate total weight
total_weight = sum([gauge['voteWeight'] for gauge in arb_gauges.values()])
arb_gauge_distributions = {}
for gauge_addr, gauge_data in arb_gauges.items():
    # Calculate distribution based on vote weight and total weight
    to_distribute = ARBITRUM_TO_DISTRIBUTE * gauge_data['voteWeight'] / total_weight
    # Cap distribution at 10%
    to_distribute = to_distribute if to_distribute < VOTE_CAP else VOTE_CAP
    arb_gauge_distributions[gauge_addr] = {
        'gaugeAddress': gauge_addr,
        'pool': gauge_data['pool'],
        'symbol': gauge_data['symbol'],
        'voteWeight': gauge_data['voteWeight'],
        'distribution': to_distribute if to_distribute < VOTE_CAP else VOTE_CAP,
        'distributionInPercent': to_distribute / ARBITRUM_TO_DISTRIBUTE * 100
    }

arb_gauge_distributions_df = pd.DataFrame.from_dict(arb_gauge_distributions, orient='index')
arb_gauge_distributions_df = arb_gauge_distributions_df.sort_values(by='distribution', ascending=False)
display(HTML(arb_gauge_distributions_df.to_html(index=False)))

gaugeAddress,pool,symbol,voteWeight,distribution,distributionInPercent
0x8135d6AbFd42707A87A7b94c5CFA3529f9b432AD,0x32dF62dc3aEd2cD6224193052Ce665DC18165841,RDNT-WETH,0.821489,9740.400000,10.000000
0x4944b07977A42C15c6a06CF4e204e24c60564104,0xCba9Ff45cfB9cE238AfDE32b0148Eb82CbE63562,rETH-bb-a-WETH-BPT,0.454235,9740.400000,10.000000
0xa8D974288Fe44ACC329D7d7a179707D27Ec4dd1c,0xc9f52540976385A84BF416903e1Ca3983c539E34,50tBTC-50WETH,0.474528,9740.400000,10.000000
0xDf464348c4EC2Bf0e5D6926b9f707c8e02301adf,0x36bf227d6BaC96e2aB1EbB5492ECec69C691943f,B-wstETH-WETH-Stable,0.937519,9740.400000,10.000000
0x175407b4710b5A1cB67a37C76859F17fb2ff6672,0xc7FA3A3527435720f0e2a4c1378335324dd4F9b3,55auraBal-45wsteth,0.777957,9740.400000,10.000000
0xc4b6cc9A444337b1Cb8cBbDD9de4d983f609C391,0x542F16DA0efB162D20bF4358EfA095B70A100f9E,2BTC,0.233378,5153.140280,5.290481
0x25869f277f474FA9459F40F5D4cB0D6A8Ab41967,0x8bc65Eed474D1A00555825c91FeAb6A8255C2107,DOLA/USDC BPT,0.210626,4650.759897,4.774711
0xacE0D479040231e3c6b17479cFd4444182d521d4,0x567ECfCB22205D279BB8Eed3E066989902bF03D5,DOLA/bbaUSD-BPT,0.210626,4650.759897,4.774711
0x0EDF6cDd81BC3471C053341B7D8Dfd1Cb367AD93,0xb3028Ca124B80CFE6E9CA57B70eF2F0CCC41eBd4,50MAGIC-50USDC,0.183482,4051.398147,4.159376
0xd758454BDF4Df7Ad85f7538DC9742648EF8e6d0A,0x8d333f82e0693f53fA48c40d5D4547142E907e1D,80PAL-20OHM,0.091146,2012.556269,2.066195
